### Using Optuna to find the best hyperparameters for our XGBoost model
The usage of a GPU NVIDIA 1050 ti for training reduces the training time by 10-40 times!

In [21]:
import numpy as np
import pandas as pd

import sys
sys.path.append('..')
from utils import get_train_data, custom_preprocessor, submit_test, train_test_split_temporal, preparation


In [15]:
X, y = get_train_data() # X comes out prepared with added columns

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split_temporal(X, y)

X_train = X_train.drop(columns=["date"])
X_valid = X_valid.drop(columns=["date"])


Loading school holidays from cache...


In [16]:
import optuna
from xgboost import XGBRegressor, DMatrix
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import time

# Example preprocessing pipeline
preprocessor = custom_preprocessor
dtrain = DMatrix(X_train, label=y_train)
dvalid = DMatrix(X_valid, label=y_valid)

# Objective function for Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "tree_method": "hist",
        "device": "cuda",
    }

    # Define the pipeline
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", XGBRegressor(**params))
    ])

    # Train and validate
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    elapsed_time = time.time() - start_time

    # Predictions on validation data
    y_pred = pipeline.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))

    # Log elapsed time for transparency
    trial.set_user_attr("elapsed_time", elapsed_time)

    return -rmse  # Optuna maximizes the objective; use negative MSE for minimization


In [17]:
# Progress monitoring callback
def progress_callback(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value:.4f} "
          f"and parameters: {trial.params}.")
    print(f"Elapsed time for trial: {trial.user_attrs['elapsed_time']:.2f} seconds")

# Create and optimize the study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, callbacks=[progress_callback])


[I 2024-12-10 17:33:10,337] A new study created in memory with name: no-name-105ec23e-bb94-46be-927d-456f80f04292


[I 2024-12-10 17:33:14,908] Trial 0 finished with value: -0.7332365863821515 and parameters: {'n_estimators': 118, 'max_depth': 5, 'learning_rate': 0.031188835851574323, 'subsample': 0.7822570220948954, 'colsample_bytree': 0.7309844026160839, 'gamma': 2.6772370931731784, 'min_child_weight': 6}. Best is trial 0 with value: -0.7332365863821515.


Trial 0 finished with value: -0.7332 and parameters: {'n_estimators': 118, 'max_depth': 5, 'learning_rate': 0.031188835851574323, 'subsample': 0.7822570220948954, 'colsample_bytree': 0.7309844026160839, 'gamma': 2.6772370931731784, 'min_child_weight': 6}.
Elapsed time for trial: 4.28 seconds


[I 2024-12-10 17:33:20,952] Trial 1 finished with value: -0.6785751489759037 and parameters: {'n_estimators': 284, 'max_depth': 6, 'learning_rate': 0.03316946030864061, 'subsample': 0.715163531364129, 'colsample_bytree': 0.7697004317751642, 'gamma': 4.220675053168738, 'min_child_weight': 6}. Best is trial 1 with value: -0.6785751489759037.


Trial 1 finished with value: -0.6786 and parameters: {'n_estimators': 284, 'max_depth': 6, 'learning_rate': 0.03316946030864061, 'subsample': 0.715163531364129, 'colsample_bytree': 0.7697004317751642, 'gamma': 4.220675053168738, 'min_child_weight': 6}.
Elapsed time for trial: 5.90 seconds


...

Trial 197 finished with value: -0.6739 and parameters: {'n_estimators': 135, 'max_depth': 9, 'learning_rate': 0.047726359206314774, 'subsample': 0.7408948271423198, 'colsample_bytree': 0.8038388592389791, 'gamma': 0.9800930517213347, 'min_child_weight': 4}.
Elapsed time for trial: 5.92 seconds


[I 2024-12-10 17:55:56,920] Trial 198 finished with value: -0.6817325319501574 and parameters: {'n_estimators': 71, 'max_depth': 9, 'learning_rate': 0.040898053334433485, 'subsample': 0.6938652692458462, 'colsample_bytree': 0.7566038779942466, 'gamma': 0.42456029349612145, 'min_child_weight': 2}. Best is trial 158 with value: -0.6634079722826679.


Trial 198 finished with value: -0.6817 and parameters: {'n_estimators': 71, 'max_depth': 9, 'learning_rate': 0.040898053334433485, 'subsample': 0.6938652692458462, 'colsample_bytree': 0.7566038779942466, 'gamma': 0.42456029349612145, 'min_child_weight': 2}.
Elapsed time for trial: 3.99 seconds


[I 2024-12-10 17:56:02,657] Trial 199 finished with value: -0.6754754455744488 and parameters: {'n_estimators': 123, 'max_depth': 9, 'learning_rate': 0.060925164242268154, 'subsample': 0.7322203535749203, 'colsample_bytree': 0.7654957236925577, 'gamma': 1.0856484922810303, 'min_child_weight': 1}. Best is trial 158 with value: -0.6634079722826679.


Trial 199 finished with value: -0.6755 and parameters: {'n_estimators': 123, 'max_depth': 9, 'learning_rate': 0.060925164242268154, 'subsample': 0.7322203535749203, 'colsample_bytree': 0.7654957236925577, 'gamma': 1.0856484922810303, 'min_child_weight': 1}.
Elapsed time for trial: 5.28 seconds


In [18]:
# Best parameters
print("Best parameters:", study.best_params)


Best parameters: {'n_estimators': 111, 'max_depth': 9, 'learning_rate': 0.0377327465507813, 'subsample': 0.7113692785668871, 'colsample_bytree': 0.8110918886927729, 'gamma': 1.4750773858086286, 'min_child_weight': 2}


Best parameters: {'n_estimators': 170, 'max_depth': 8, 'learning_rate': 0.0361177159793649, 'subsample': 0.5555986573425598, 'colsample_bytree': 0.7840311734222667, 'gamma': 4.57284935770335, 'min_child_weight': 5}

In [19]:
# Re-train the final model with the best parameters
final_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(**study.best_params, tree_method="hist", device="cuda"))
])
final_pipeline.fit(X_train, y_train)

# Evaluate on test set
y_test_pred = final_pipeline.predict(X_valid)
final_mse = mean_squared_error(y_valid, y_test_pred)
print("Final MSE on test set:", np.round(np.sqrt(final_mse), 2))


Final MSE on test set: 0.66


In [20]:
submit_test(final_pipeline, "XGBoost_optuna")

Loading school holidays from cache...


In [28]:

data = pd.read_parquet("../data/final_test.parquet")
file_name = "../submissions/" + "XGBoost_optuna" + "_submission.csv"

X_test = preparation(data)
X_test = X_test.drop(columns=["date"])

y_predict = final_pipeline.predict(X_test)

results = pd.DataFrame(
dict(
    Id=np.arange(y_predict.shape[0]),
    log_bike_count=y_predict,
    )
)
results.to_csv(file_name, index=False)

Loading school holidays from cache...


The public score is 0.89. I have clerly overfitted during optuna...